In [ ]:
import json, tempfile, shutil
from pathlib import Path
import sys
import cv2
import numpy as np
import torch
from PIL import Image, ImageFont, ImageDraw
from IPython.display import display

# ─────────────── setup imports ───────────────
project_root = Path(".").resolve().parent  # adjust if needed
sys.path.insert(0, str(project_root))

from utils.owl   import load_owl, detect_with_owl
from utils.sam   import load_sam, segment_with_sam, post_process_sam_masks
from utils.mask  import overlay_masks_on_image, create_black_white_mask
from utils.box_mask import draw_annotated
from utils.resize import resize_long_side
from lama.runner import run_lama

In [ ]:
image_path = Path("../pipeline_optimization_dataset/1SIka2FSC_tE6_94GW3GsRvb-Gi5CA8wa__Küche_Wohnung1.jpg")  
image = Image.open(image_path).convert("RGB")

In [ ]:
owl_thresh  = 0.30
target_long = 2048

In [ ]:
processor_owl, model_owl = load_owl()
processor_sam, model_sam = load_sam()

In [ ]:
text_labels = [[
    "a house number", "a license plate", "person", "a face",
    "a religious symbol", "a political symbol", "a cat", "a dog",
    "a calendar", "a paper", "a framed picture", "a picture",
    "a poster board", "a name", "a sex toy", "a nude image",
]]

font = ImageFont.truetype("/Library/Fonts/Arial.ttf", size=50)

In [ ]:
boxes_p, scores, labels = detect_with_owl(image, text_labels, processor_owl, model_owl, threshold=owl_thresh)
raw_boxes = [b.tolist() for b in boxes_p]

In [ ]:
annotated_det = draw_annotated(image.copy(), raw_boxes, [float(s.item()) for s in scores], labels, font=font)
display(annotated_det)

In [ ]:
tb = torch.tensor(raw_boxes, dtype=torch.float32).unsqueeze(0)
outs, ins = segment_with_sam(image, tb, processor_sam, model_sam)
masks = post_process_sam_masks(outs, processor_sam, ins)[0]

In [ ]:
overlay = overlay_masks_on_image(image, masks, overlay_color=(255, 0, 0, 128), dilation_px=0)
display(overlay)

In [ ]:
bw_mask = create_black_white_mask(
    masks, threshold=0.5, combine=True, dilation_px=0
)
display(bw_mask)

In [ ]:
img_np = np.array(image)
img_rs  = resize_long_side(img_np, target_long)
mask_rs = resize_long_side(np.array(bw_mask.convert("L")), target_long,nearest=True)

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    tmpdir = Path(tmpdir)
    tmp_img  = tmpdir / "img.png"
    tmp_mask = tmpdir / "mask.png"
    Image.fromarray(img_rs).save(tmp_img, "PNG")
    cv2.imwrite(str(tmp_mask), mask_rs)

    lama_model_dir = Path("../lama/big-lama")

    try:
        result_png = run_lama(
            image_path=tmp_img,
            mask_path=tmp_mask,
            model_dir=lama_model_dir,
            out_dir=tmpdir,
        )
        result_img = Image.open(result_png).convert("RGB")
        display(result_img)
    except Exception as e:
        print(f"→ LaMa failed: {e}")